In [1]:
import pandas as pd

movies = pd.read_csv('movies.csv', sep=',')
ratings = pd.read_csv('ratings.csv', sep=',')

### Rekomendacje dla użytkownika o id=600, top 10

In [6]:
from collections import Counter

# Pobranie ocen użytkownika 600
user_id = 600
user_ratings = ratings[ratings['userId'] == user_id]

In [7]:
# Połączenie ocen z informacjami o filmach
user_movies = user_ratings.merge(movies, on='movieId')

# Wyodrębnienie najczęściej ocenianych gatunków przez użytkownika
all_genres = user_movies['genres'].str.split('|').sum()
top_genres = [genre for genre, _ in Counter(all_genres).most_common(5)]
top_genres_set = set(top_genres)

In [8]:
# Filtrowanie filmów, które pasują do ulubionych gatunków
def has_common_genre(genre_str):
    return not top_genres_set.isdisjoint(set(genre_str.split('|')))

genre_matched_movies = movies[movies['genres'].apply(has_common_genre)]

In [9]:
# Usunięcie filmów już ocenionych przez użytkownika
unseen_movies = genre_matched_movies[~genre_matched_movies['movieId'].isin(user_ratings['movieId'])]

# Dołączenie ocen i obliczenie średniej + liczby ocen
ratings_for_unseen = ratings[ratings['movieId'].isin(unseen_movies['movieId'])]
aggregated = ratings_for_unseen.groupby('movieId').agg(
    średnia_ocen=('rating', 'mean'),
    rating_count=('rating', 'count')
).reset_index()

# Tylko popularne filmy (min. 50 ocen)
popular = aggregated[aggregated['rating_count'] >= 50]

recommendations = popular.merge(movies, on='movieId')

In [10]:
# Sortowanie i wybór TOP 10
top_10 = recommendations.sort_values(by='średnia_ocen', ascending=False).head(10).reset_index(drop=True)
top_10.insert(0, 'Top 10', range(1, 11))
top_10_final = top_10[['Top 10', 'movieId', 'title', 'genres', 'średnia_ocen']]

print(f'Top 10 rekomendacji dla użytkownika o ID = {user_id}:\n')
print(top_10_final.to_string(index=False))

Top 10 rekomendacji dla użytkownika o ID = 600:

 Top 10  movieId                                  title                           genres  średnia_ocen
      1     1276                  Cool Hand Luke (1967)                            Drama      4.271930
      2      904                     Rear Window (1954)                 Mystery|Thriller      4.261905
      3     1221         Godfather: Part II, The (1974)                      Crime|Drama      4.259690
      4    48516                   Departed, The (2006)             Crime|Drama|Thriller      4.252336
      5       50             Usual Suspects, The (1995)           Crime|Mystery|Thriller      4.237745
      6     1208                  Apocalypse Now (1979)                 Action|Drama|War      4.219626
      7     1252                       Chinatown (1974) Crime|Film-Noir|Mystery|Thriller      4.211864
      8     1193 One Flew Over the Cuckoo's Nest (1975)                            Drama      4.203008
      9     1089        

###Znajdz 10 rekomendacji po obejrzaniu filmu: "Toy Story"

In [3]:
# Zdefiniowanie docelowych gatunków
target_genres = {'Animation', 'Children'}

# Filtrowanie filmów zawierających przynajmniej jeden z docelowych gatunków
def has_target_genres(genre_string):
    genres = set(genre_string.split('|'))
    return not target_genres.isdisjoint(genres)

filtered_movies = movies[movies['genres'].apply(has_target_genres)]

In [4]:
genre_ratings = ratings.merge(filtered_movies, on='movieId')

avg_ratings = genre_ratings.groupby('movieId').agg(
    średnia_ocen=('rating', 'mean')
).reset_index()

# Dołączenie tytułów i gatunków
movie_info = filtered_movies[['movieId', 'title', 'genres']].drop_duplicates()
merged = avg_ratings.merge(movie_info, on='movieId')

In [5]:
# Filtrowanie filmów z co najmniej 50 ocenami
rating_counts = genre_ratings.groupby('movieId').size().reset_index(name='rating_count')
merged = merged.merge(rating_counts, on='movieId')
popular = merged[merged['rating_count'] >= 50]

top_10 = popular.sort_values(by='średnia_ocen', ascending=False).head(10).reset_index(drop=True)

top_10.insert(0, 'Top 10', range(1, 11))

top_10_final = top_10[['Top 10', 'movieId', 'title', 'genres', 'średnia_ocen']]

print('Top 10 rekomendacji po obejrzeniu "Toy Story":\n')
print(top_10_final.to_string(index=False))

Top 10 rekomendacji po obejrzeniu "Toy Story":

 Top 10  movieId                                                title                                           genres  średnia_ocen
      1     5618 Spirited Away (Sen to Chihiro no kamikakushi) (2001)                      Adventure|Animation|Fantasy      4.155172
      2    78499                                   Toy Story 3 (2010) Adventure|Animation|Children|Comedy|Fantasy|IMAX      4.109091
      3    60069                                        WALL·E (2008)      Adventure|Animation|Children|Romance|Sci-Fi      4.057692
      4     1148          Wallace & Gromit: The Wrong Trousers (1993)                  Animation|Children|Comedy|Crime      4.035714
      5      953                         It's a Wonderful Life (1946)                   Children|Drama|Fantasy|Romance      4.034483
      6    68954                                            Up (2009)               Adventure|Animation|Children|Drama      4.004762
      7     2804     